In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim
import torch.nn.modules as nn
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder: Increased capacity with higher channel counts
        self.encoder1 = self.conv_block(3, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder
        self.upconv4 = self.upconv(1024, 512)
        self.decoder4 = self.conv_block(1024, 512)
        self.upconv3 = self.upconv(512, 256)
        self.decoder3 = self.conv_block(512, 256)
        self.upconv2 = self.upconv(256, 128)
        self.decoder2 = self.conv_block(256, 128)
        self.upconv1 = self.upconv(128, 64)
        self.decoder1 = self.conv_block(128, 64)

        # Final output layer
        self.final = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )

        # Max-pooling for down-sampling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def conv_block(self, in_channels, out_channels):
        """Convolutional block: Two conv layers with BatchNorm and ReLU."""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def upconv(self, in_channels, out_channels):
        """Transposed convolution for up-sampling."""
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        # Encoder
        enc1 = self.encoder1(x)  # Output: 64 channels
        enc2 = self.encoder2(self.pool(enc1))  # Output: 128 channels
        enc3 = self.encoder3(self.pool(enc2))  # Output: 256 channels
        enc4 = self.encoder4(self.pool(enc3))  # Output: 512 channels

        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))  # Output: 1024 channels

        # Decoder
        dec4 = self.decoder4(torch.cat((self.upconv4(bottleneck), enc4), dim=1))  # Output: 512 channels
        dec3 = self.decoder3(torch.cat((self.upconv3(dec4), enc3), dim=1))  # Output: 256 channels
        dec2 = self.decoder2(torch.cat((self.upconv2(dec3), enc2), dim=1))  # Output: 128 channels
        dec1 = self.decoder1(torch.cat((self.upconv1(dec2), enc1), dim=1))  # Output: 64 channels

        # Final depth map
        out = self.final(dec1)
        return out

In [ ]:
import h5py
import random
# prepare data
class TemporalConsistencyDataset(torch.utils.data.Dataset):
    def __init__(self, images, depths):
        super(TemporalConsistencyDataset, self).__init__()
        self.images = images
        self.depths = depths

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, i):
        image = torch.tensor(self.images[i]) / 255.0
        depth = torch.tensor(np.expand_dims(self.depths[i], axis=0))
        return image, depth


#/content/gdrive/MyDrive/ECE_1508_project/nyu_depth_v2_labeled.mat
print("Loading dataset for temporal consistency training...")
with h5py.File('/content/gdrive/MyDrive/ECE_1508_project/nyu_depth_v2_labeled.mat', 'r') as f:
    images = f['images'][:]
    depths = f['depths'][:]



Loading dataset for temporal consistency training...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/gdrive/MyDrive/ECE_1508_project/nyu_depth_v2_labeled.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
depths = depths / 10
test_num = 200
batch_size = 4
valid_dataset = TemporalConsistencyDataset(images[:test_num], depths[:test_num])

# Create DataLoader
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

NameError: name 'depths' is not defined

In [ ]:
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Mean Absolute Error (MAE)
def mae(pred, target):
    return torch.mean(torch.abs(pred - target))

# Root Mean Squared Error (RMSE)
def rmse(pred, target):
    return torch.sqrt(torch.mean((pred - target) ** 2))

# Peak Signal-to-Noise Ratio (PSNR)
def psnr(pred, target):
    mse = torch.mean((pred - target) ** 2)
    max_pixel = 1.0  # Since output uses Sigmoid, normalized to [0, 1]
    return 10 * torch.log10(max_pixel**2 / mse)

# Structural Similarity Index (SSIM)
def ssim_metric(pred, target):
    """
    Calculate Structural Similarity Index (SSIM)
    """
    # Convert output from [B, 1, H, W] to [H, W] format
    pred_np = pred.cpu().detach().numpy().squeeze()  # [B, 1, H, W] -> [H, W]
    target_np = target.cpu().detach().numpy().squeeze()

    # Calculate win_size
    min_side = min(pred_np.shape)  # Get minimum side length of image
    win_size = min(min_side, 7)  # win_size cannot exceed minimum side length, max is 7

    # If image size is less than 7x7, set win_size to minimum side length
    if min_side < 7:
        win_size = min_side if min_side % 2 == 1 else min_side - 1  # Make win_size odd

    # Calculate SSIM
    return ssim(pred_np, target_np, data_range=target_np.max() - target_np.min(), win_size=win_size)

# Evaluate model metrics
def evaluate_metrics(model, dataloader, device):
    model.eval()  # Set to evaluation mode

    mae_total = 0
    rmse_total = 0
    psnr_total = 0
    ssim_total = 0
    count = 0  # Used to calculate batch averages

    with torch.no_grad():  # No gradient computation needed during evaluation
        for (images1, images2), (depth1, depth2) in dataloader:
            images1, images2 = images1.to(device), images2.to(device)
            depth1, depth2 = depth1.to(device), depth2.to(device)

            # Forward pass
            outputs1 = model(images1)
            outputs2 = model(images2)

            # Calculate metrics for each batch
            mae_total += mae(outputs1, depth1) + mae(outputs2, depth2)
            rmse_total += rmse(outputs1, depth1) + rmse(outputs2, depth2)
            psnr_total += psnr(outputs1, depth1) + psnr(outputs2, depth2)

            # Calculate SSIM (for each output-target pair)
            try:
                ssim_total += ssim_metric(outputs1, depth1) + ssim_metric(outputs2, depth2)
            except ValueError as e:
                # Skip batch if calculation error occurs (e.g. image size too small)
                print(f"Skipping SSIM calculation due to: {e}")

            count += 1

    # Calculate averages
    mae_avg = mae_total / count
    rmse_avg = rmse_total / count
    psnr_avg = psnr_total / count
    ssim_avg = ssim_total / count if count > 0 else 0  # If there are valid SSIM calculations

    return mae_avg, rmse_avg, psnr_avg, ssim_avg

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import tqdm

# Load the trained model
def load_trained_model(model_path, device):
    model = UNet()  # Initialize model
    model.load_state_dict(torch.load(model_path))  # Load trained model parameters
    model.eval()  # Set to evaluation mode
    model.to(device)  # Move model to specified device (GPU/CPU)
    return model

# Specify device (use GPU if available, otherwise use CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Validation phase
valid_model_path = '/content/gdrive/MyDrive/ECE_1508_project/supervised_edge_aware_temporal_smooth_1_epoch_21.pth'  # Replace with your model path
valid_model = load_trained_model(valid_model_path, device)
valid_model.eval()

# Initialize metrics
mae_avg, rmse_avg, psnr_avg, ssim_avg = 0, 0, 0, 0

with torch.no_grad():

    # Calculate validation metrics
    mae_avg, rmse_avg, psnr_avg, ssim_avg = evaluate_metrics(valid_model, valid_loader, device)

print(f"Validation Metrics - MAE: {mae_avg:.4f}, RMSE: {rmse_avg:.4f}, PSNR: {psnr_avg:.4f}, SSIM: {ssim_avg:.4f}")

<ipython-input-3-330e75e30dc8>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load trained model parameters


KeyboardInterrupt: 

# Predict Video

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms

# Load trained model
def load_trained_model(model_path, device):
    model = UNet()  # Initialize model
    model.load_state_dict(torch.load(model_path))  # Load trained model parameters
    model.eval()  # Set to evaluation mode
    model.to(device)  # Move model to specified device (GPU/CPU)
    return model

# Preprocess function: Convert image from BGR to RGB and normalize, ensure consistent size
def preprocess_image(image, target_size=(256, 256)):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb = cv2.resize(image_rgb, target_size)  # Resize to model's required input size
    image_rgb = np.transpose(image_rgb, (2, 0, 1))  # Convert to (C, H, W) format
    image_rgb = torch.tensor(image_rgb, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    image_rgb /= 255.0  # Normalize to [0, 1]
    return image_rgb

# Visualize predictions
def visualize_predictions(pred_depth1, pred_depth2, frame_idx):
    pred_depth1 = pred_depth1.squeeze().cpu().detach().numpy()  # Convert to 2D (H, W)
    pred_depth2 = pred_depth2.squeeze().cpu().detach().numpy()  # Convert to 2D (H, W)

    # Normalize depth maps to visualization range (0 to 255)
    depth_map1 = (pred_depth1 - np.min(pred_depth1)) / (np.max(pred_depth1) - np.min(pred_depth1)) * 255
    depth_map2 = (pred_depth2 - np.min(pred_depth2)) / (np.max(pred_depth2) - np.min(pred_depth2)) * 255

    depth_map1 = depth_map1.astype(np.uint8)
    depth_map2 = depth_map2.astype(np.uint8)

    # Visualize as pseudo-color image
    depth_map1_colored = cv2.applyColorMap(depth_map1, cv2.COLORMAP_JET)
    depth_map2_colored = cv2.applyColorMap(depth_map2, cv2.COLORMAP_JET)

    # Display images using matplotlib
    if frame_idx % 10 == 0:
        print(f"Visualizing depth for frame {frame_idx}")

        # Display first depth map
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title(f"Predicted Depth - Frame {frame_idx}")
        plt.imshow(depth_map1_colored)
        plt.axis('off')

        # Display second depth map
        plt.subplot(1, 2, 2)
        plt.title(f"Predicted Depth - Frame {frame_idx + 1}")
        plt.imshow(depth_map2_colored)
        plt.axis('off')

        # Update display
        plt.show()

def save_depth_video(pred_depths, output_path, frame_size, fps=30):
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Try using XVID codec
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use MP4V codec
    width, height = frame_size
    out_video = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

    print(f"Frame size: {frame_size}")
    print(f"Number of frames: {len(pred_depths)}")

    first_depth = pred_depths[0].cpu().numpy()
    print(f"Depth map shape: {first_depth.shape}")
    print(f"Depth range: {np.min(first_depth)} to {np.max(first_depth)}")

    for depth in pred_depths:
        depth_map = depth.cpu().numpy()
        depth_map = (depth_map - np.min(depth_map)) / (np.max(depth_map) - np.min(depth_map)) * 255
        depth_map = depth_map.astype(np.uint8)
        depth_map = cv2.resize(depth_map, (width, height))
        depth_map_colored = cv2.applyColorMap(depth_map, cv2.COLORMAP_JET)

        # Write to video
        out_video.write(depth_map_colored)

    out_video.release()
    print(f"Depth video saved to {output_path}")

# Main function for video depth estimation
def test_video_depth_estimation(video_path, model, device, output_video_path=None):
    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    # Get video width, height and fps
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Store predicted depth maps
    pred_depths = []

    # Process video frame by frame
    prev_frame_rgb = None
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Video finished

        # Preprocess image
        current_frame_rgb = preprocess_image(frame)
        current_frame_rgb = current_frame_rgb.to(device)

        if prev_frame_rgb is not None:
            # Estimate depth for both frames
            with torch.no_grad():
                pred_depth1 = model(prev_frame_rgb)  # Depth map for first frame
                pred_depth2 = model(current_frame_rgb)  # Depth map for second frame

            # Visualize predicted depth maps
            visualize_predictions(pred_depth1, pred_depth2, frame_idx)

            # Save predicted depth maps
            pred_depths.append(pred_depth1.squeeze().cpu())  # Store first frame's depth map
            pred_depths.append(pred_depth2.squeeze().cpu())  # Store second frame's depth map

            frame_idx += 1

        # Update previous frame
        prev_frame_rgb = current_frame_rgb

    cap.release()

    # Save output video if path is specified
    if output_video_path is not None:
        save_depth_video(pred_depths, output_video_path, (frame_width, frame_height), fps)

# Specify device (use GPU if available, otherwise use CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load trained model
model_path = '/content/gdrive/MyDrive/ECE_1508_project/supervised_edge_aware_temporal_smooth_1_epoch_21.pth'  # Replace with your model path
model = load_trained_model(model_path, device)

# Specify video path
video_path = '/content/gdrive/MyDrive/ECE_1508_project/test1.mp4'  #

# Specify output video path
output_video_path = '/home/ub/Documents/Al/model_final/save/output_depth_video1.mp4'  # Can be set to None if you don't want to save video



<ipython-input-2-03d382149dd3>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load trained model parameters


FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/ECE_1508_project/supervised_edge_aware_temporal_smooth_1_epoch_21.pth'

In [3]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
def temporal_consistency(pred_frame_t, pred_frame_t1):
    """
    Computes temporal consistency (MSE) between consecutive predicted frames.

    Args:
        pred_frame_t (torch.Tensor): Predicted depth map at time t [1, H, W].
        pred_frame_t1 (torch.Tensor): Predicted depth map at time t+1 [1, H, W].

    Returns:
        float: Temporal consistency loss (MSE).
    """
    return torch.nn.functional.mse_loss(pred_frame_t, pred_frame_t1).item()

# Visualization function
def visualize_comparison(rgb_frame, pred_depth1, pred_depth2, frame_idx):
    """
    Visualize the comparison between two predicted depth maps.

    Args:
        rgb_frame (np.array): Original RGB frame.
        pred_depth1 (np.array): Predicted depth map from model 1.
        pred_depth2 (np.array): Predicted depth map from model 2.
        frame_idx (int): Current frame index.
    """
    pred_depth2 /= 10
    plt.figure(figsize=(12, 6))

    # Show input frame
    plt.subplot(1, 3, 1)
    plt.imshow(rgb_frame)
    plt.title(f"Frame {frame_idx}")
    plt.axis("off")

    # Show prediction from model 1
    plt.subplot(1, 3, 2)
    plt.imshow(pred_depth1, cmap='plasma')
    plt.title("Model 1 Prediction")
    plt.axis("off")

    # Show prediction from model 2
    plt.subplot(1, 3, 3)
    plt.imshow(pred_depth2, cmap='plasma')
    plt.title("Model 2 Prediction")
    plt.axis("off")

    # Save or show frame
    plt.savefig(f"comparison_frame_{frame_idx}.png")
    plt.close()

In [4]:
# Compare two models on a video
def compare_depth_models_no_ground_truth(model1, model2, video_path, device="cuda"):
    """
    Compares two depth estimation models on a video without ground truth.

    Args:
        model1 (torch.nn.Module): First depth estimation model.
        model2 (torch.nn.Module): Second depth estimation model.
        video_path (str): Path to the input video.
        device (str): Device to run the models on.

    Returns:
        dict: Temporal consistency metrics for both models.
    """
    # Initialize metrics
    temporal_metrics_model1 = []
    temporal_metrics_model2 = []

    # Open video file
    cap = cv2.VideoCapture(video_path)
    prev_depth1, prev_depth2 = None, None
    frame_idx = 0
    i = 0
    # Process each frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        print(i)
        i+=1

        # Preprocess frame (convert to tensor and normalize)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame_tensor = torch.from_numpy(rgb_frame).float().permute(2, 0, 1).unsqueeze(0).to(device) / 255.0

        # Predict depth using both models
        pred_depth1 = model1(rgb_frame_tensor)
        pred_depth2 = model2(rgb_frame_tensor)

        # Resize predictions to match input frame
        pred_depth1 = torch.nn.functional.interpolate(pred_depth1, size=rgb_frame_tensor.shape[-2:], mode='bilinear', align_corners=False)
        pred_depth2 = torch.nn.functional.interpolate(pred_depth2, size=rgb_frame_tensor.shape[-2:], mode='bilinear', align_corners=False)

        # Compute temporal consistency if not the first frame
        if prev_depth1 is not None and prev_depth2 is not None:
            temporal_metrics_model1.append(temporal_consistency(prev_depth1, pred_depth1))
            temporal_metrics_model2.append(temporal_consistency(prev_depth2, pred_depth2))

        # Update previous frame predictions
        prev_depth1 = pred_depth1.clone()
        prev_depth2 = pred_depth2.clone()

        # Visualize results (Optional)
        visualize_comparison(rgb_frame, pred_depth1[0, 0].detach().cpu().numpy(), pred_depth2[0, 0].detach().cpu().numpy(), frame_idx)

        frame_idx += 1

    # Release video capture
    cap.release()

    # Compute average temporal consistency
    avg_temporal_model1 = np.mean(temporal_metrics_model1)
    avg_temporal_model2 = np.mean(temporal_metrics_model2)

    return {"Model1_Temporal": avg_temporal_model1, "Model2_Temporal": avg_temporal_model2}

In [5]:
class UNet_old(nn.Module):
    def __init__(self):
        super(UNet_old, self).__init__()
        # encoder
        self.encoder1 = self.conv_block(3, 32)
        self.encoder2 = self.conv_block(32, 64)
        self.encoder3 = self.conv_block(64, 128)
        self.encoder4 = self.conv_block(128, 256)
        # decoder
        self.decoder1 = self.conv_block(256, 128)
        self.decoder2 = self.conv_block(128, 64)
        self.decoder3 = self.conv_block(64, 32)
        # one chnnel for depth estimation
        self.final = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.Conv2d(16, 1, kernel_size=3, padding=1),
        )

        # up-sample and down-sample
        self.pool = nn.MaxPool2d(2, 2)
        self.up1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up3 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))

        enc4 = self.encoder4(self.pool(enc3))

        dec1 = self.decoder1(torch.cat((self.up1(enc4), enc3), dim=1))
        dec2 = self.decoder2(torch.cat((self.up2(dec1), enc2), dim=1))
        dec3 = self.decoder3(torch.cat((self.up3(dec2), enc1), dim=1))
        out = self.final(dec3)

        return out

In [6]:
# Load models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = UNet_old()
checkpoint = torch.load("/content/gdrive/MyDrive/ECE_1508_project/epoch_149.pth")
model1.load_state_dict(checkpoint)
model1.to(device)
model1.eval()

model2 = UNet()
checkpoint = torch.load("/content/gdrive/MyDrive/ECE_1508_project/supervised_edge_aware_temporal_e_30.pth")
model2.load_state_dict(checkpoint)
model2.to(device)
model2.eval()
# Path to the video
video_path = "/content/gdrive/MyDrive/ECE_1508_project/test1.mp4"


<ipython-input-6-b102a171f70f>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/gdrive/MyDrive/ECE_1508_project/epoch_149.pth")
<ipython-in

In [7]:
# Compare models
metrics = compare_depth_models_no_ground_truth(model1, model2, video_path, device="cuda")
print("Temporal Consistency Metrics:")
print(metrics)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27